In [47]:
import re
import polars as pl
import altair as alt
from math import cos, radians

from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.theme.enable('irozhlas')
alt.data_transformers.disable_max_rows()

C:\Users\micha\AppData\Local\Temp\ipykernel_19740\4053582104.py:13: AltairDeprecationWarning: 
Deprecated since `altair=5.5.0`. Use altair.theme instead.
Most cases require only the following change:

    # Deprecated
    alt.themes.enable('quartz')

    # Updated
    alt.theme.enable('quartz')

If your code registers a theme, make the following change:

    # Deprecated
    def custom_theme():
        return {'height': 400, 'width': 700}
    alt.themes.register('theme_name', custom_theme)
    alt.themes.enable('theme_name')

    # Updated
    @alt.theme.register('theme_name', enable=True)
    def custom_theme():
        return alt.theme.ThemeConfig(
            {'height': 400, 'width': 700}
        )

See the updated User Guide for further details:
    https://altair-viz.github.io/user_guide/api.html#theme
    https://altair-viz.github.io/user_guide/customization.html#chart-themes
  alt.themes.register('irozhlas', kristi_promin)


DataTransformerRegistry.enable('default')

In [48]:
df = pl.scan_parquet(
    '../zpravodajske-weby/data/dny/*.parquet'
).collect(
).sort(
    by='cas'
).with_columns(
    pl.col("titulek").str.replace(" nad ","_nad_").str.replace("České ","České_").str.replace("Staré ","Staré_").str.replace("Starého ","Starého_").str.replace("Nové ","Nové_").str.replace("Nového ","Nového_").str.replace("Moravské ","Moravské_").str.replace("Českých ","Českých_").str.replace("Moravských ","Moravských_").str.replace("Nebezpečná"," ").str.replace("KrimiMartin"," ").str.replace("\xa0"," ")
).with_columns(
    pl.col('cas').dt.ordinal_day().alias('den')
).filter(
    pl.col("cas").dt.year() == 2025
)

In [49]:
", ".join(df.group_by("web").len().sort(by="len",descending=True).select(pl.col("web")).to_series().to_list())

'seznamzpravy.cz, blesk.cz, irozhlas.cz, denikn.cz, novinky.cz, denik.cz, idnes.cz, parlamentnilisty.cz, tn.nova.cz, aktualne.cz, lidovky.cz, echo24.cz, ceskenoviny.cz, extra.cz, forum24.cz, drbna.cz, zpravy.idnes.cz, cnn.iprima.cz'

In [50]:
def mesto(titulek, spojky=['z','ze']):
    titulek = re.sub(r'[^\w\s]', '', titulek)
    for s in spojky:
        if f" {s} " in titulek:
            fraze = f"{s} " + titulek.split(f' {s} ')[1].split(' ')[0]
            if fraze != fraze.lower():
                return fraze.replace("_"," ")

In [51]:
def najdi_vyskyty(popisek, retezec):
    return df.filter(
        pl.col("cas").dt.year() == 2025
    ).filter(
        pl.col("titulek").str.contains(retezec)
    ).unique(
        subset="titulek",keep='first'
    ).with_columns(
        pl.col('titulek').map_elements(mesto,return_dtype=pl.String).alias('mesto')
    ).group_by(
        'mesto'
    ).agg(
        pl.col('den').unique().len()
    ).sort(
        by='den',descending=True
    ).drop_nulls(
    ).with_columns(
        pl.lit(popisek).alias("co")
    )

In [52]:
do_grafu_df = pl.concat(
    [
        najdi_vyskyty("rekordman",r"(?i)rekordman(a|em|ovi|ům|y|i|ech|ům){0,1} ze{0,1} "),
        najdi_vyskyty("muž",r"(?i)muž(e|i|em|ích|ům|ovi|ů){0,1} ze{0,1} "),
        najdi_vyskyty("mladík",r"(?i)mladí(k|ka|kovi|ku|kem|ci|ky|ků|kům|kách|cích){0,1} ze{0,1} "),
        najdi_vyskyty("lidé",r"(?i)lid(é|í|mi|em|ech) ze{0,1} "),
        najdi_vyskyty("dítě",r"(?i)d(ěti|ětí|ětem|ětech|ítě|ítětem|ítěti|ítěte) ze{0,1} "),
        najdi_vyskyty("žena",r"(?i)žen(a|y|u|ě|ou|ami|ách|ám){0,1} ze{0,1} "),
        najdi_vyskyty("dívka",r"(?i)dív(ka|kou|ce|ek|ky|kám|kách|kami) ze{0,1} "),
        najdi_vyskyty("stařík","(?i)staří(k|ka|kovi|ku|kem|ci|ky|kům|kách|cích){0,1} ze{0,1} "),
        najdi_vyskyty("rodačka",r"(?i)rodač(ka|ky|kou|kách|ce|kami){0,1} ze{0,1} "),
        najdi_vyskyty("důchodce",r"(?i)důchodc(e|i|em|ovi|ům|ích){0,1} ze{0,1} "),
        najdi_vyskyty("senior",r"(?i)senio[rř](a|em|i|u|ovi|r|y|ům|ech){0,1} ze{0,1} "),
        najdi_vyskyty("studentka",r"(?i)student[kc](a|ou|e|ek|y|ám|ami|ách){0,1} ze{0,1} "),
        najdi_vyskyty("manželé",r"(?i)manžel(é|ům|y|ech|ové) ze{0,1} "),
        najdi_vyskyty("kamarádi",r"(?i)kamarád(i|ům|ech|ů) ze{0,1} "),
        najdi_vyskyty("student",r"(?i)student(ovi|a|i|ům|y|ech|em){0,1} ze{0,1} "),
        najdi_vyskyty("násilník",r"(?i)násilní[kc](a|em|u|ovi|i|y|ách|ích){0,1} ze{0,1} "),
        najdi_vyskyty("vědkyně",r"(?i)vědky[ňn](i|e|ě|ím|í|ěmi|ích){0,1} ze{0,1} "),
        najdi_vyskyty("vědec",r"(?i)věd(ec|ci|cům|cem|ců|ce|cích){1} ze{0,1} "),
        najdi_vyskyty("zahrádkář",r"(?i)zahrádkář(ovi|e|i|e|em|ům|ích){0,1} ze{0,1} "),
        najdi_vyskyty("houbař",r"(?i)houbař(i|e|em|ovi|ům|ích){0,1} ze{0,1} "),
        najdi_vyskyty("podivín",r"(?i)podivín(a|em|y|ech|i|ům|ovi){0,1} ze{0,1} "),
        najdi_vyskyty("bezdomovec",r"(?i)bezdomov(ec|ci|covi|ce|cem|ci|cům|cích){0,1} ze{0,1} "),
        najdi_vyskyty("seniorka",r"(?i)senior[kc](a|e|y|ou|ám|ách|ami){0,1} ze{0,1} "),
        najdi_vyskyty("důchodkyně",r"(?i)důchodk(yně|yni|yní|yněmi|yních){0,1} ze{0,1} "),
        najdi_vyskyty("školák",r"(?i)školá[kc](a|em|u|ovi|r|y|ům|ech|ích){0,1} ze{0,1} "),
        najdi_vyskyty("školačka",r"(?i)školač[kc](a|ou|y|ami|ách|ek){0,1} ze{0,1} "),
        najdi_vyskyty("přátelé",r"(?i)přátel(e|é|i|ům|ích|ech|y){0,1} ze{0,1} "),
        najdi_vyskyty("pár",r"(?i)[^\w]pár(em|y|u|ům|ech){0,1} ze{0,1} "),
        najdi_vyskyty("chlapec",r"(?i)chlap(ec|ci|cem|covi|cích|cům|ců) ze{0,1} "),
        najdi_vyskyty("výtržník",r"(?i)výtržník(a|c|y|en|ou|e|em|ovi|i|a|ou|y|ovi|ům|ech|y|e|i|í|ích){0,1} ze{0,1} "),
        najdi_vyskyty("podvodník",r"(?i)podvodník(a|em|ou|u|ovi|i|ům|ech|y|ích){0,1} ze{0,1} "),
        najdi_vyskyty("cyklista",r"(?i)cyklist(a|u|ou|y|i|ům|ech){0,1} ze{0,1} "),
        najdi_vyskyty("řidička",r"(?i)řidič[kc](e|a|ou|y|ám|ami|ách){0,1} ze{0,1} "),
        najdi_vyskyty("řidič",r"(?i)řidič(e|i|em|ovi|ům|ích|){0,1} ze{0,1} "),
        najdi_vyskyty("dealer",r"(?i)deale[rř](a|em|u|ovi|i|ům|ech|y|ích){0,1} ze{0,1} "),
        najdi_vyskyty("rodák",r"(?i)rodá[kc](a|em|u|ovi|ů|i|ům|y|ích){0,1} ze{0,1} "),
        najdi_vyskyty("lékař",r"(?i)lékař(e|i|em|u|ovi|ům|y|ů|ích){0,1} ze{0,1} "),
        najdi_vyskyty("doktor",r"(?i)dokto[rř](a|em|u|ovi|i|ech|ům|y|ích){0,1} ze{0,1} "),
        najdi_vyskyty("lékařka",r"(?i)lékař(ka|kou|ce|ky|kami|ek|kách){0,1} ze{0,1} "),
        najdi_vyskyty("doktorka",r"(?i)doktor(ka|kou|ce|ky|kami|ek|kách){0,1} ze{0,1} ")
    ]
)

In [53]:
nechceme = [
    'z ANO',
    'z Hoří',
    'z Hradu',
    'z Aljašky',
    'z Ukrajiny',
    'z Jemenu',
    'z ODS',
    'ze Světových',
    'z ČR',
    'ze Slovenska',
    'z Česka',
    'z Kyjeva',
    'z USA',
    'z US',
    'z Německa',
    'z Gazy',
    'z Ruska',
    'z Bohnic'
]

do_grafu = do_grafu_df.filter(
    ~pl.col("mesto").is_in(nechceme)
).sort(by="den",descending=True)

vyber = do_grafu.group_by("mesto").agg(pl.col("den").sum()).sort(by="den",descending=True).head(25).select(pl.col("mesto")).to_series().to_list()

do_grafu = do_grafu.filter(pl.col("mesto").is_in(vyber))

filtr_cetnosti = (
    do_grafu
    .group_by("co")
    .agg(
        # We explicitly name this "pocet_mest"
        pl.col("mesto").unique().len().alias("pocet_mest") 
    )
    .filter(pl.col("pocet_mest") >= 3)
    .select(pl.col("co"))
    .to_series()
    .to_list()
)

do_grafu = do_grafu.filter(pl.col("co").is_in(filtr_cetnosti))

aspon_x_lidi = do_grafu.group_by("mesto").agg(pl.col("co").unique().len()).filter(pl.col("co") >= 3).select(pl.col("mesto")).to_series().to_list()

do_grafu = do_grafu.filter(pl.col("mesto").is_in(aspon_x_lidi))

poradi_kategorii = do_grafu.group_by("co").agg(pl.col("den").sum()).sort(by="den",descending=True).select(pl.col("co")).to_series().to_list()

In [54]:
do_grafu.sample(4)

mesto,den,co
str,u32,str
"""z Olomoucka""",3,"""žena"""
"""z Mostu""",1,"""manželé"""
"""z Hradce""",6,"""muž"""
"""z Brněnska""",1,"""seniorka"""


In [55]:
poradi_kategorii

['muž',
 'žena',
 'mladík',
 'rodák',
 'vědec',
 'dívka',
 'chlapec',
 'senior',
 'seniorka',
 'student',
 'lékař',
 'manželé',
 'dítě',
 'studentka',
 'školák',
 'důchodce',
 'rodačka',
 'důchodkyně',
 'lidé',
 'lékařka',
 'řidič']

In [56]:
mrizka='lightgray'

base = alt.Chart(
    do_grafu,
    width=500,
    title=alt.Title(
        "V kolika různých dnech se loni objevili v novém titulku na zpravodajských webech…",
        subtitle=[
            "Počítají se všechny pády jednotného i množného čísla.",
            "Sledované stránky: seznamzpravy.cz, blesk.cz, irozhlas.cz, denikn.cz, novinky.cz, denik.cz,",
            "idnes.cz, parlamentnilisty.cz, tn.nova.cz, aktualne.cz, lidovky.cz, echo24.cz, ceskenoviny.cz,",
            "extra.cz, forum24.cz, drbna.cz, zpravy.idnes.cz (pouze od 1. 7.), cnn.iprima.cz"
        ],
        align="left",
        anchor="start"
    )
).encode(
    alt.Y("co:N", sort=poradi_kategorii, title=None, axis=alt.Axis(grid=True, gridColor=mrizka, tickColor=mrizka, tickOffset=0, tickWidth=0.5, gridWidth=0.5, zindex=0, domainColor="white")),
    alt.X("mesto:N", sort=vyber, title=None, axis=alt.Axis(orient='top', gridColor=mrizka, tickColor=mrizka, tickOffset=0, tickWidth=0.5, gridWidth=0.5,labelAngle=305, domainColor="white",grid=True, zindex=0)),
)

# 2. Create the rectangle layer
rects = base.mark_rect().encode(
    alt.Color("den:Q", scale=alt.Scale(range=['#FFFFE4',"#D6534B"], type="linear"), legend=None, title=None)
)

# 3. Create the text layer
# We map the "text" channel to the "len" column
text = base.mark_text(baseline='middle').encode(
    text='den:Q',
    # You can conditionally change color if the background is too dark
    color=alt.value('black') 
)

# 4. Combine them
chart = rects + text

chart = chart.configure_view(stroke='transparent')

chart

alt.LayerChart(...)

In [81]:
me_to_neurazi(
    graf=chart, 
    kredity="",
    soubor="03_lide_z"
)

<figure>
    <a href="https://data.irozhlas.cz/grafy/03_lide_z.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy/03_lide_z.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [57]:
df.filter(pl.col('titulek').str.contains(r"(?i)muž(e|i|em|ích|ům|ovi|ů){0,1} ze{0,1} brna")).group_by(pl.col("cas").dt.month()).agg(pl.col("titulek").unique().len()).sort(by="cas")

cas,titulek
i8,u32
1,2
2,1
3,1
5,6
6,6
7,6
8,3
9,3
10,1


In [58]:
df.filter(pl.col('titulek').str.contains(r"(?i)muž(e|i|em|ích|ům|ovi|ů){0,1} ze{0,1} brna")).select(pl.col("titulek")).unique().to_series().to_list()

['Brněnská DrbnaLegendární muž z Brna se letos činil. Mrštil gyrosem po věrozvěstovi nebo močil z radniční věže',
 'Muž z Brna slavil tak divoce, že skončil s rozbitou hlavou ve žlabu u kolejí',
 'Muž z Brna se bál, že má napadený účet. Tak radši vybral 900\xa0tisíc a\xa0dal je na ulici neznámému člověku',
 'Muž z Brna si do nemocnice přinesl cihlu a vysklil dveře. Skončil na psychiatrii',
 'Muži z Brna se podle policie vloupali do 115 vozů. Skončili ve vazbě',
 'Muže z Brna naštvala přítelkyně. Policistů se ptal, jaký trest dostane za její vraždu',
 'Muž z Brna vypil v obchodě půl lahve rumu a ustlal si mezi regály',
 'Brno se nedá urazit. Na vtipy o svém městě jsou i hrdí, říká autorka Muže z Brna',
 'Chci zabít svou přítelkyni, kolik za to dostanu? Brněnské policisty vyděsil šokující telefonát  VávraNa známou policejní linku 158 volají lidé s nejrůznějšími příběhy i dotazy. Telefonát pětatřicetiletého muže z Brna se však vymykal…',
 'Brno se nedá urazit. Na vtipy o svém městě jsou i

In [59]:
df.filter(pl.col('titulek').str.contains(r"(?i)žen(a|y|u|ě|ou|ami|ách|ám){0,1} ze{0,1} šumpers")).select(pl.col("titulek")).unique().to_series().to_list()

['Úmrtí v Česku na malárii: Co nezanedbat před cestou do tropů, jaké jsou příznakyČesko v posledních dnech obletěla zpráva, že starší žena ze Šumperska, která strávila krátký čas na východoafrickém…',
 'Chtěla investovat, ale naletěla. Žena ze Šumperska přišla o téměř čtyři miliony',
 'Žena ze Šumperska přišla o 215\xa0tisíc, sedla na lep falešným policistům a\xa0bankéřům',
 'Slepě se zamilovala na sociální síti, podvodník ženu ze Šumperska připravil o statisíce',
 'Žena ze Šumperska zemřela na malárii:',
 'Úmrtí v Česku na malárii: Co nezanedbat před cestou do tropů, jaké jsou příznakyČesko v posledních dnech obletěla zpráva, že starší žena ze Šumperska, která…',
 'Ženu ze Šumperska zlákali na výhru v soutěži. Obrali ji o\xa0víc, než jí nabízeli',
 'Ženu ze Šumperska oklamali online podvodníci:',
 'Úspory skončily v rukou podvodníků. Ženy ze Šumperska naletěly falešným potomkům']

In [60]:
df.filter(pl.col('titulek').str.contains(r"(?i)muž(e|i|em|ích|ům|ovi|ů){0,1} ze{0,1} žižkova")).select(pl.col("titulek")).unique().to_series().to_list()

[]

In [61]:
df.filter(pl.col('titulek').str.contains(r"(?i)muž(e|i|em|ích|ům|ovi|ů){0,1} ze{0,1} smíchova")).select(pl.col("titulek")).unique().to_series().to_list()

[]

In [62]:
len(df.filter(pl.col('titulek').str.contains(r"(?i)muž(e|i|em|ích|ům|ovi|ů){0,1} v{0,1} praze")).select(pl.col("titulek")).unique().to_series().to_list())

57

In [63]:
len(df.filter(pl.col('titulek').str.contains(r"(?i)muž(e|i|em|ích|ům|ovi|ů){0,1} v{0,1} brně")).select(pl.col("titulek")).unique().to_series().to_list())

22